In [0]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score,confusion_matrix

In [0]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [0]:
traindata=pd.read_csv('train.csv')
itemdata=pd.read_csv('item_data.csv')
#viewlog=pd.read_csv('view_log.csv')

In [0]:
testdata=pd.read_csv('test.csv')

In [4]:
traindata.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,15-11-2018 00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,15-11-2018 00:01,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,15-11-2018 00:02,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,15-11-2018 00:02,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,15-11-2018 00:02,62873,473,latest,0,0


In [5]:
itemdata.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [23]:
testdata.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G
0,a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1
1,caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,latest,0
2,13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,latest,1
3,39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,latest,1
4,bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,latest,1


In [0]:
#data1=pd.merge(traindata,viewlog)

In [0]:
#data2=pd.merge(data1,itemdata)

In [0]:
#data2.head()

In [0]:
#data3=data2.drop(['is_click'],axis=1)
#data3.head()

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
traindata["app_code"] = le.fit_transform(traindata["app_code"])
traindata["user_id"] = le.fit_transform(traindata["user_id"])
traindata["os_version"] = le.fit_transform(traindata["os_version"])
traindata["is_4G"] = le.fit_transform(traindata["is_4G"])

In [0]:
#le = LabelEncoder()
#testdata["app_code"] = le.fit_transform(testdata["app_code"])
#testdata["user_id"] = le.fit_transform(testdata["user_id"])

In [0]:
testdata["app_code"] = le.fit_transform(testdata["app_code"])
testdata["user_id"] = le.fit_transform(testdata["user_id"])
testdata["os_version"] = le.fit_transform(testdata["os_version"])
testdata["is_4G"] = le.fit_transform(testdata["is_4G"])

In [0]:
x=traindata.iloc[:,2:6].values
y=traindata.iloc[:,6].values

In [0]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
dt_model = DecisionTreeClassifier(criterion= 'entropy', random_state = 0, class_weight={1:9,0:1}, max_depth=100, min_samples_split=200, 
                                  )
dt_model.fit(x_train, y_train)
y_pred_test = dt_model.predict(x_test)
print("Accuracy score for model ", accuracy_score(y_test, y_pred_test))
confusion_mx = pd.DataFrame(confusion_matrix(y_test, y_pred_test))
print(confusion_mx)
probs = dt_model.predict_proba(x_test) 
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
print("Precision score for DT model ", precision_score(y_test, y_pred_test))
print("Recall score for DT model ", recall_score(y_test, y_pred_test))

Accuracy score for model  0.8355638604110903
       0     1
0  48776  7957
1   1811   859
AUC: 0.67
Precision score for DT model  0.09743647912885663
Recall score for DT model  0.32172284644194754


In [0]:
test_data_X =testdata.iloc[:,2:]

In [27]:
test_data_X.head()

,user_id,app_code,os_version,is_4G
0,16423,94,1,1
1,10858,39,1,0
2,9213,211,1,1
3,8391,150,1,1
4,13031,171,1,1


In [0]:
test_data_pred = dt_model.predict(test_data_X)
submission = pd.DataFrame({'impression_id':testdata.iloc[:,0], 'is_click':test_data_pred})
submission.to_csv('submission_dt.csv', index=False)